Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0003, window_size=5
Epoch [1052/1500], Training Loss: 1.4935169181005978, Validation Loss: 4.272566318511963
Early stopping at epoch 1052
Final Test Loss: 4.418914318084717

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


def split_data_with_window(x_in, y_in, split_window_size):
    # Initialize lists to store training and temporary sets
    x_out1_list, y_out1_list, x_out2_list, y_out2_list = [], [], [], []

    # Iterate through the data with the specified window size
    for i in range(0, len(x_in) - split_window_size, split_window_size + 1):
        x_out1_out2 = x_in.iloc[i:i+split_window_size+1]
        y_out1_out2 = y_in.iloc[i:i+split_window_size+1]

        # Separate the last row for the temporary set
        # [ :-1]: all elements except the last one
        # [-1:]:  selects only the last element
        # (:) is used to indicate slicing of a sequence
        # sequence[start : end : step]

        x_out1 = x_out1_out2.iloc[:-1]
        y_out1 = y_out1_out2.iloc[:-1]

        x_out2 = x_out1_out2.iloc[-1:]
        y_out2 = y_out1_out2.iloc[-1:]

        x_out1_list.append(x_out1)
        y_out1_list.append(y_out1)
        x_out2_list.append(x_out2)
        y_out2_list.append(y_out2)

    # Concatenate the lists into pandas DataFrames
    x_out1 = pd.concat(x_out1_list)
    y_out1 = pd.concat(y_out1_list)
    x_out2 = pd.concat(x_out2_list)
    y_out2 = pd.concat(y_out2_list)

    return x_out1, y_out1, x_out2, y_out2










# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)



# Split Data to train and temp

# Define your split_window_size
split_window_size = 3

# Call the split_data_with_window function
x_train, y_train, x_temp, y_temp = split_data_with_window(x_data, y_data, split_window_size)


"""

# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))

"""

# Split temp into val and test

# Define your split_window_size
split_window_size = 1

# Call the split_data_with_window function
x_val, y_val, x_test, y_test = split_data_with_window(x_temp, y_temp, split_window_size)




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""









class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [9]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [1]
learning_rates = [0.0004]
window_sizes = [5]

num_epochs = 1500
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0004, window_size=5


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1500], Training Loss: 27156.800065458516, Validation Loss: 26517.798828125
Epoch [2/1500], Training Loss: 25032.448395751853, Validation Loss: 24625.51171875
Epoch [3/1500], Training Loss: 23433.21921403852, Validation Loss: 23156.517578125
Epoch [4/1500], Training Loss: 22027.5540707363, Validation Loss: 21823.076171875
Epoch [5/1500], Training Loss: 20739.042967078512, Validation Loss: 20551.982421875
Epoch [6/1500], Training Loss: 19590.729650406956, Validation Loss: 19423.87109375
Epoch [7/1500], Training Loss: 18543.016668294258, Validation Loss: 18398.404296875
Epoch [8/1500], Training Loss: 17584.287421004166, Validation Loss: 17468.224609375
Epoch [9/1500], Training Loss: 16702.483996535906, Validation Loss: 16617.76953125
Epoch [10/1500], Training Loss: 15891.403394818359, Validation Loss: 15820.7783203125
Epoch [11/1500], Training Loss: 15145.29850130117, Validation Loss: 15061.609375
Epoch [12/1500], Training Loss: 14455.926694789987, Validation Loss: 14368.72949218

In [ ]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [1]
learning_rates = [0.0004]
window_sizes = [5]

num_epochs = 1500
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0007, window_size=5
Epoch [1/1500], Training Loss: 25411.24711494686, Validation Loss: 24401.83984375
Epoch [2/1500], Training Loss: 22565.914460869244, Validation Loss: 22063.27734375
Epoch [3/1500], Training Loss: 20315.269413412538, Validation Loss: 19922.8046875
Epoch [4/1500], Training Loss: 18425.450800612343, Validation Loss: 18147.189453125
Epoch [5/1500], Training Loss: 16816.49951734048, Validation Loss: 16608.201171875
Epoch [6/1500], Training Loss: 15425.173289274886, Validation Loss: 15265.5595703125
Epoch [7/1500], Training Loss: 14219.047912826038, Validation Loss: 14024.4755859375
Epoch [8/1500], Training Loss: 13160.80661821665, Validation Loss: 12980.7783203125
Epoch [9/1500], Training Loss: 12220.211974309908, Validation Loss: 12113.783203125
Epoch [10/1500], Training Loss: 11386.808369850125, Validation Loss: 11349.4111328125
Epoch [11/1500], Training Loss: 10645.47638226688, Validation Loss

KeyboardInterrupt: 

In [10]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [1]
learning_rates = [0.0001]
window_sizes = [5]

num_epochs = 1500
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0001, window_size=5
Epoch [1/1500], Training Loss: 29180.910660816964, Validation Loss: 29115.841796875
Epoch [2/1500], Training Loss: 28490.621125741116, Validation Loss: 28476.634765625
Epoch [3/1500], Training Loss: 27875.5025118435, Validation Loss: 27871.70703125
Epoch [4/1500], Training Loss: 27291.745596651857, Validation Loss: 27296.8671875
Epoch [5/1500], Training Loss: 26737.937201601657, Validation Loss: 26751.525390625
Epoch [6/1500], Training Loss: 26213.70474501326, Validation Loss: 26235.421875
Epoch [7/1500], Training Loss: 25718.683968091846, Validation Loss: 25748.134765625
Epoch [8/1500], Training Loss: 25252.437326470175, Validation Loss: 25289.18359375
Epoch [9/1500], Training Loss: 24814.36263721805, Validation Loss: 24857.95703125
Epoch [10/1500], Training Loss: 24403.736375581386, Validation Loss: 24453.6875
Epoch [11/1500], Training Loss: 24019.70402562207, Validation Loss: 24075.513671

In [11]:
l

NameError: name 'l' is not defined

In [ ]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [1]
learning_rates = [0.0005]
window_sizes = [5]

num_epochs = 1500
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0005, window_size=5


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1500], Training Loss: 26554.435056082148, Validation Loss: 25769.255859375
Epoch [2/1500], Training Loss: 24113.030602345916, Validation Loss: 23648.970703125
Epoch [3/1500], Training Loss: 22248.190985589346, Validation Loss: 21938.279296875
Epoch [4/1500], Training Loss: 20633.63550542519, Validation Loss: 20363.251953125
Epoch [5/1500], Training Loss: 19226.18263190864, Validation Loss: 19010.71875
Epoch [6/1500], Training Loss: 17970.956462144102, Validation Loss: 17823.931640625
Epoch [7/1500], Training Loss: 16842.529355600494, Validation Loss: 16732.078125
Epoch [8/1500], Training Loss: 15824.234414015571, Validation Loss: 15738.044921875
Epoch [9/1500], Training Loss: 14907.033740300398, Validation Loss: 14790.0
Epoch [10/1500], Training Loss: 14076.092330914087, Validation Loss: 13944.2060546875
Epoch [11/1500], Training Loss: 13315.717384574731, Validation Loss: 13203.388671875
Epoch [12/1500], Training Loss: 12618.84352988923, Validation Loss: 12527.7255859375
Epoch

In [ ]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [1]
learning_rates = [0.0003]
window_sizes = [5]

num_epochs = 1500
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=1, learning_rate=0.0003, window_size=5


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1500], Training Loss: 27787.00012770133, Validation Loss: 27317.6796875
Epoch [2/1500], Training Loss: 26060.76742158675, Validation Loss: 25744.07421875
Epoch [3/1500], Training Loss: 24657.189663854282, Validation Loss: 24423.05859375
Epoch [4/1500], Training Loss: 23502.63231663645, Validation Loss: 23337.021484375
Epoch [5/1500], Training Loss: 22523.0760909937, Validation Loss: 22371.47265625
Epoch [6/1500], Training Loss: 21527.508691539293, Validation Loss: 21407.619140625
Epoch [7/1500], Training Loss: 20594.548296384204, Validation Loss: 20486.337890625
Epoch [8/1500], Training Loss: 19730.691050508074, Validation Loss: 19634.634765625
Epoch [9/1500], Training Loss: 18928.973166281237, Validation Loss: 18843.11328125
Epoch [10/1500], Training Loss: 18180.33244681142, Validation Loss: 18105.09375
Epoch [11/1500], Training Loss: 17478.64647459988, Validation Loss: 17415.5078125
Epoch [12/1500], Training Loss: 16818.69883499127, Validation Loss: 16770.716796875
Epoch [13